In [3]:
# standard ds imports
import numpy as np
import pandas as pd

# for wrangling and exploration
# import wrangle
# import explore
# import model

# for visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# filter out noise
import warnings
warnings.filterwarnings('ignore')

# MYERS BRIGGS TWITTER ASSESSMENT

This project is designed to analyze the relationship between the 16 Myers Briggs personality types and twitter behavior. The results.....

----

## Executive Summary:

Goals:
- stuff
- things

Key Findings:
- stuff
- things

Takeaways:
- stuff
- things

Recommendations:
- stuff
- things

----
## 1. Planning
 - Create deliverables:
     - README
     - final_report.ipynb
     - working_report.ipynb
     - canva presentation
 - Bring over functional wrangle.py, explore.py, and model.py files
 - Acquire the data from the Kaggle and Twitter via acquire.py's web scraping. Save the data locally.
 - Prepare and split the data via the prepare.py functions
 - Explore the data and define hypothesis. Run the appropriate statistical tests in order to accept or reject each null hypothesis. Document findings and takeaways.
 - Model a baseline in predicting personality type and document the accuracy.
 - Fit and train classification models to predict personality type or domain on the train dataset.
 - Evaluate the models by comparing the train and validation data.
 - Select the best model and evaluate it on the train data.
 - Develop and document all findings, takeaways, recommendations and next steps.

----
## 2. Data Acquisition
- stuff
- things

#### Data Dictionary

| Target | Type | Description |
| ---- | ---- | ---- |
| type | str | the 16 myers briggs personality types |

| Feature Name | Type | Description |
| ---- | ---- | ---- |
|  |  |  |

### Acquisition Takeaways
- stuff
- things

----
## 2. Data Preparation
- stuff
- things

In [4]:
# prepare

In [5]:
# split

In [6]:
# verify the split

### Preparation Takeaways
- stuff
- things

----
## 2. Data Exploration

What does the data look like? (Percent population -> flow chart with ratios)

### Question 1: What scale of groupings shows a significant relationship in sentiment? Pairs, types, or domains?

In [ ]:
# where is there a significant difference in samples?

#### Hypothesis 1:

    Ho ->
    Ha ->
    T-Test, One-Sample, Two-Tailed

### Question 1 Takeaways
- stuff
- things

---
### Question 2a: What words are seen across all personality types?

#### Hypothesis 2:

    Ho ->
    Ha ->
    T-Test, One-Sample, Two-Tailed

### Question 2b: What words are unique to personality types?

### Question 2 Takeaways
- stuff
- things

---
### Question 3: Is there a relationship between word count and personality type?

#### Hypothesis 3:

    Ho ->
    Ha ->
    T-Test, One-Sample, Two-Tailed

### Question 3 Takeaways
- stuff
- things

### Exploration Takeaways
- stuff
- things
- more stuff

---
## Data Modeling
The goal is to maximize accuracy.

| Features Dropped | Features Kept |
| ---- | ---- |
| customer_id | contract_type |

In [1]:
# modeling results based on looking at the data in 3 different pools

### Modeling Takeaways:
- stuff
- things

---
## Conclusion
Main conclusion note here

### Recommendations
- stuff
- things

### Next Steps:
- stuff
- things
---